In [1]:
import sys
sys.path.append("..")

import os
cwd = os.getcwd()

import torch
import numpy as np
import glob

from GENUINE.data.datasets import PatchDataset
from torch.utils.data import DataLoader
from GENUINE.utils.device import best_gpu
from GENUINE.utils.visualize import gridPlot
from GENUINE.data.custom_transforms import ToTensor
from GENUINE.utils.get_data import _zenodo_get

_zenodo_get("10.5072/zenodo.1171221", sandbox=True)

DATA_PATH = os.path.join(os.path.expanduser('~'), ".GENUINE")
DATASET = glob.glob(os.path.join(DATA_PATH, "*.h5"))[0]
MODELS = glob.glob(os.path.join(DATA_PATH, "*.model"))

Found existing Baseline.model in /home/simon_g/.GENUINE
Found existing GENUINE_B.model in /home/simon_g/.GENUINE
Found existing GENUINE_E.model in /home/simon_g/.GENUINE
Found existing GENUINE.model in /home/simon_g/.GENUINE
Found existing RetinaNet.model in /home/simon_g/.GENUINE
Found existing TEST_DATASET_SMALL.h5 in /home/simon_g/.GENUINE


['/home/simon_g/.GENUINE/RetinaNet.model',
 '/home/simon_g/.GENUINE/GENUINE.model',
 '/home/simon_g/.GENUINE/GENUINE_B.model',
 '/home/simon_g/.GENUINE/Baseline.model',
 '/home/simon_g/.GENUINE/GENUINE_E.model']

In [ ]:
ds = PatchDataset(DATASET, dataset="TEST", transform=[ToTensor()], n=10)
dl = DataLoader(ds, batch_size=8)

device = best_gpu(verbose=False)

for m in MODELS:
    
    if "RetinaNet" in m:
        continue
    
    model = torch.load(m, map_location="cpu")
    model.eval()
    
    print(model.__class__)
    
    model.to(device)
    
    preds, patches, targets = [], [], []
    for X, y in dl:
        
        preds.extend((model(X.to(device)) > 0).int().squeeze().tolist())
        patches.append(X.numpy().transpose(0, 2, 3, 1))
        targets.extend(y.tolist())
        
    gridPlot(np.vstack(patches), labels=preds, targets=targets, sz=(3,3))